In [2]:
import os

import tiktoken
from dotenv import load_dotenv
from openai import OpenAI

In [3]:
load_dotenv(".env")
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [ ]:
prompt = "write a dad joke"

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    max_tokens=100,
    temperature=0.9,
)

completion = completion.to_dict()

In [ ]:
completion["choices"][0]["message"]["content"]

### What is a prompt?

* The **input** you provide, which is typically a **text**, but it could also be an **image** or an **audio** file. 
* It serves as a set of **instructions** the **model** uses to predict the desired response.

### What is prompt engineering?

* The process of **discovering prompts** that reliably returns useful or desired results.
* Also known as **instructions-tuning**

<div style="text-align: center; color: red;">Average prompts >  average responses!</div>

### What is a model?

* The OpenAI API offers a diverse set of [models](https://platform.openai.com/docs/models) with different capabilities and [pricing](https://openai.com/api/pricing/).

* Each model is updated frequently. If you don't specify the version of the model, you will be using the `latest` version. 

* To specify the model version, we can suffix it to the base model name. For example, to use the `2024-07-18` version of `gpt-4o-mini`, we should use: `gpt-4o-mini-2024-07-18`. 

* To find all the available versions of a model, search for that model. For example: [gpt-4o-mini](https://platform.openai.com/docs/models#gpt-4o-mini).

<span style="color:green">
Task: Find the version of the model we used
</span>

In [ ]:
completion["..."]

### What is a token?


Tokens can be thought of as pieces of words. These tokens are not cut up exactly where the words start or end - tokens can include trailing spaces and even sub-words. 

Here are some helpful rules of thumb for understanding tokens in terms of lengths:

* 1 token ~= 4 chars in English
* 1 token ~= ¾ words
* 100 tokens ~= 75 words

<span style="color:green">
Task: Find the number of tokens in the above example for:

1. The prompt that you sent
2. The completion that you received
3. The total
</span>

In [ ]:
completion["usage"]["..."]

In [ ]:
completion["usage"]["..."]

In [ ]:
completion["usage"]["..."]

<span style="color:green">
Task: can you confirm these numbers using the OpenAI 
<a href="https://platform.openai.com/tokenizer" style="color:blue">tokenizer</a> UI?
</span>

<span style="color:green">
Task: can you use
<a href="https://github.com/openai/tiktoken" style="color:blue">tiktoken</a> library to count the number of prompt tokens programmatically?
</span>

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
tokens = encoding.encode(...)
len(...)

#### Token Limits

Depending on the [model](https://platform.openai.com/docs/models) used, requests can use up to 128,000 tokens shared between prompt and completion.

#### Token Pricing

Requests to different models are priced differently. You can find details on token pricing [here](https://openai.com/api/pricing/). 

### What is temperature?

* You can think of temperature like randomness, with 0 being least random (most deterministic) and 2 being most random (least deterministic). 

* When using low values for temperature (e.g. 0.2) the model responses will tend to be more consistent but may feel more robotic. 

* Values higher than 1.0, especially values close to 2.0, can lead to erratic model outputs. 

* If your goal is creative outputs, a combination of a slightly higher than normal temperature (e.g. 1.2) combined with a prompt specifically asking the model to be creative may be your best bet, but you should experiment!

<span style="color:green">
Task: Change the temperature above and resubmit
</span>

In [ ]:
prompt = "write a dad joke and be creative"

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    max_tokens=100,
    temperature=...,
)

completion = completion.to_dict()

completion["choices"][0]["message"]["content"]

<span style="color:green">
Task: Create a <span style="color:blue">get_completion</span> function to send a prompt to a specific model and to return the response content
</span>

In [11]:
def get_completion(prompt, model="gpt-4o-mini"):
    completion = ...

    completion = completion.to_dict()

    content = completion["choices"][0]["message"]["content"]

    return content

## Image Generation

<a href="https://platform.openai.com/docs/models#dall-e" style="color:blue">DALL·E</a> can create realistic images and art from a prompt, edit an existing image, or create variations of a provided image.

Currently, there are 2 variations: DALL·E 2 and DALL·E 3 with different options for generating images. <a href="https://platform.openai.com/docs/guides/images?language=python#which-model-should-i-use" style="color:blue">Which model should you use?</a>

In [ ]:
response = client.images.generate(
    model="dall-e-2",
    prompt="a white siamese cat",
    size="1024x1024",
    n=1,
)

response = response.to_dict()

response["data"][0]["url"]

`model`: The model to use for image generation. Must be one of `dall-e-2` or `dall-e-3`

`prompt`: description of the desired image(s). Maximum length is 1000 characters for `dall-e-2` and 4000 characters for `dall-e-3`

`size`: size of the generated images. Must be one of `256x256`, `512x512`, or `1024x1024` for `dall-e-2`. Must be one of `1024x1024`, `1792x1024`, or `1024x1792` for `dall-e-3 models`.

`n`: number of images to generate. Must be between 1 and 10. For `dall-e-3`, only n=1 is supported.

<span style="color:green">
Task: Play with the prompt above to generate different images.
</span>

#### Edits (DALL·E 2 only)

* Edit or extend an image by uploading an image and mask indicating which areas should be replaced.

* This process is also known as <b>inpainting</b>.

* The transparent areas of the mask indicate where the image should be edited, and the prompt should describe the full new image, <b>not just the erased area</b>.

* The uploaded image and mask must both be square PNG images, less than 4MB in size, and have the same dimensions as each other. 

* The non-transparent areas of the mask aren't used to generate the output, so they don’t need to match the original image.

* There're free tools to create a mask, for example 
<a href="https://onlinepngtools.com/erase-part-of-png" style="color:blue">Onlinepngtools</a>

In [ ]:
response = client.images.edit(
    model="dall-e-2",
    image=open("data/sunlit_lounge.png", "rb"),
    mask=open("data/mask.png", "rb"),
    prompt="A sunlit indoor lounge area with a pool containing a flamingo",
    size="1024x1024",
    n=1,
)

response = response.to_dict()

response["data"][0]["url"]

<span style="color:green">
Task: Edit a different image using different masks. 
</span>

#### Variations (DALL·E 2 only)

* Generate a variation of a given image.

* The input image must be a square PNG image less than 4MB in size.

In [ ]:
response = client.images.create_variation(
    model="dall-e-2",
    image=open("data/corgi_and_cat_paw.png", "rb"),
    size="1024x1024",
    n=1,
)

response = response.to_dict()

response["data"][0]["url"]

<span style="color:green">
Task: Generate variations of a different image. 
</span>

#### DALL·E 3 Prompting

* The model takes in your prompt and automatically rewrites it:
    * For safety reasons
    * To add more detail (more detailed prompts generally result in higher quality images)

* The updated prompt is visible in the `revised_prompt` field of the data response object.

* You can't disable this feature, but you can get outputs closer to your requested image by adding the following to your prompt:

`I NEED to test how the tool works with extremely simple prompts. DO NOT add any detail, just use it AS-IS.`

In [ ]:
response = client.images.generate(
    model="dall-e-3",
    prompt="a white siamese cat",
    size="1024x1024",
    n=1,
)

response = response.to_dict()

response["data"][0]["url"]

<span style="color:green">
Task: Find the `revised_prompt` in the response. 
</span>

In [ ]:
response["data"][0]["..."]